In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scombi_do.app import main
from helpers import item_to_img_overlay, my_read_method
from pystac import STAC_IO, read_file, Item, Catalog, CatalogType
from shapely.geometry import shape
from ipyleaflet import Map

### Stage the STAC catalogs

In [3]:
input_references = ['https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414.json', 
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.json',
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720.json',
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/48/M/YT/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429.json']

In [4]:
STAC_IO.read_text_method = my_read_method
    
catalogs = []

for index, input_reference in enumerate(input_references):

    items = []
    
    thing = read_file(input_reference)

    if isinstance(thing, Item):

        items.append(thing)

    elif isinstance(thing, Catalog):

        for item in thing.get_items():

            items.append(item)

    # create catalog
    catalog = Catalog(id=items[0].id,
              description='staged STAC catalog with {}'.format(items[0].id))

    catalog.add_items(items)

    catalog.normalize_and_save(root_href=items[0].id,
                               catalog_type=CatalogType.RELATIVE_PUBLISHED)

    catalog.describe()
    
    catalogs.append(catalog.get_self_href())

2020-11-25T15:09:17 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443
2020-11-25T15:09:18 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414.json HTTP/1.1" 200 17169
2020-11-25T15:09:18 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443


* <Catalog id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
  * <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>


2020-11-25T15:09:18 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.json HTTP/1.1" 200 17119
2020-11-25T15:09:18 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443


* <Catalog id=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>
  * <Item id=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>


2020-11-25T15:09:19 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720.json HTTP/1.1" 200 17116
2020-11-25T15:09:19 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443


* <Catalog id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>
  * <Item id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>


2020-11-25T15:09:19 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/48/M/YT/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429.json HTTP/1.1" 200 20153


* <Catalog id=S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429>
  * <Item id=S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429>


In [5]:
catalogs

['/workspace/charter-pe/scombi-do/demo/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/catalog.json',
 '/workspace/charter-pe/scombi-do/demo/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/catalog.json',
 '/workspace/charter-pe/scombi-do/demo/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/catalog.json',
 '/workspace/charter-pe/scombi-do/demo/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429/catalog.json']

### Scombi-do

#### Simple RGB combination

In [6]:
expressions = ['(interp v1 (asarray 0 10000) (asarray 0 1))', 
               '(interp v2 (asarray 0 10000) (asarray 0 1))',
               '(interp v3 (asarray 0 10000) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [7]:
bands = ['red', 'green', 'blue']

Define the input local STAC catalogs

In [8]:
channel_inputs = [catalogs[0], 
                  catalogs[0],
                  catalogs[0]]
channel_inputs

In [9]:
color = 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.35' 

In [10]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [11]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['color'] = color
params['aoi'] = aoi


In [12]:
result = main(**params)

2020-11-25T15:09:20 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-11-25T15:09:20 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-11-25T15:09:20 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-11-25T15:09:42 INFO     Rescaling and COG for input assets
2020-11-25T15:09:42 INFO     Getting band red from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B04.tif
2020-11-25T15:10:36 INFO     Getting band green from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B03.tif
2020-11-25T15:11:13 INFO     Getting band blue from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B02.tif
2020-11-25T15:11:50 INFO     Build VRT
2020-11-25T15:11:50 INFO     10.0
2020-11-25T15:11:50 INF

In [14]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m

Map(center=[-35.8975, 136.88900000000004], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…

#### Normalized difference

In [15]:
expressions = ['(interp (/ (- v1 v2) (+ v1 v2)) (asarray -1 1) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [16]:
bands = ['nir', 'red']

Define the input local STAC catalogs

In [17]:
channel_inputs = [catalogs[0], 
                  catalogs[0]]
channel_inputs

In [18]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [19]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['aoi'] = aoi
params['lut'] = 'viridis'

In [20]:
result = main(**params)

2020-11-25T15:12:53 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-11-25T15:12:53 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-11-25T15:13:01 INFO     Rescaling and COG for input assets
2020-11-25T15:13:01 INFO     Getting band nir from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B08.tif
2020-11-25T15:13:40 INFO     Getting band red from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B04.tif
2020-11-25T15:13:43 INFO     Build VRT
2020-11-25T15:13:43 INFO     10.0
2020-11-25T15:13:43 INFO     Pimp me
2020-11-25T15:13:43 INFO     2
2020-11-25T15:13:43 INFO     Applying look-up table
2020-11-25T15:13:43 INFO     Adding band 1 of 3
2020-11-25T15:13:44 INFO     Adding band 2 of 3
2020-11-25T15:13:44 INFO     Adding band 3 of 3
2020-11-25T15:13:49 INFO     STAC
2020-11

In [22]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m

Map(center=[-35.8975, 136.88900000000004], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…

### Multitemporal RGB composite

In [23]:
expressions = ['(interp v1 (asarray 0 10000) (asarray 0 1))', 
               '(interp v2 (asarray 0 10000) (asarray 0 1))',
               '(interp v3 (asarray 0 10000) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [24]:
bands = ['red', 'green', 'blue']

Define the input local STAC catalogs

In [30]:
channel_inputs = [catalogs[0], 
                  catalogs[1],
                  catalogs[2]]
channel_inputs

In [31]:
color = 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45' 

In [32]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [33]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['color'] = color
params['aoi'] = aoi


In [34]:
result = main(**params)

2020-11-25T15:19:10 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-11-25T15:19:10 INFO     <Item id=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>
2020-11-25T15:19:10 INFO     <Item id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>
2020-11-25T15:19:26 INFO     Rescaling and COG for input assets
2020-11-25T15:19:26 INFO     Getting band red from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B04.tif
2020-11-25T15:19:26 INFO     Getting band green from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2020/1/S2B_53HPA_20200130_0_L2A/B03.tif
2020-11-25T15:20:15 INFO     Getting band blue from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2020/2/S2B_53HPA_20200209_0_L2A/B02.tif
2020-11-25T15:21:04 INFO     Build VRT
2020-11-25T15:21:04 INFO     10.0
2020-11-25T15:21:04 INFO 

In [35]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m

Map(center=[-35.8975, 136.88900000000004], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…